#### Name: `Jeffrey Scanlon`
#### AndrewID: `jscanlo2`

## Determining the most popular gender neutral names

The United States Social Security administration keeps records of all births and provides some of this [data](https://www.ssa.gov/oact/babynames) to the public in a file where each line is of the format:

KY,F,1912,Dorothy,209

this is to be interpreted as "In 1912, 209 female babies were born in Kentucky who were given the first name Dorothy". 

In this exercise you are to write a pyspark program that works with RDDs to determine the most popular gender neutral names.  We define a gender neutral name as a baby name that has been given to both a boy and girl baby.  We define a popular gender neutral name as a name where the ratio of the number of boys with that name to the number of girls with that name is in the range \[0.25..4\]

#### Outline of the steps involved:

1. Read in all babynames into an RDD with `Names` named tuple
2. Determine a count of all female names
3. Determine all gender neutral names by joining male_names and female_names
4. Filter the neutral names so that the ratio lies between 0.25 and 4 (inclusive)
5. Sort the filtered names in descending order by the total number of babies with that name

#### Working with groups in an RDD

Recall that our pespective is that an RDD is super-power enabled list --- resilience and distribution are builtin. As with regular Python lists elements of an RDD can in general be anything.  Some RDD operations require a _key_ e.g., `reduceByKey`, `groupByKey`, `sortByKey`.  In such instance the RDD has to have the form `[ (key, value), (key, value), ...]` i.e., the top  level has to be top of two elements--- a key and a value.  The value itself can have any structure. Hence, on occasion you may have to rearrange the elements of an RDD.  For example if you have an RDD with \[('Jack', 87), ('Jill', 92)\] and you want to sort you will need to do:
```
r.map(lambda x: ( x[1], x[0] ))  # rearrange the tuples
 .sortByValues()
```


The whole babynames file from the SSA has 6028151 records (lines) and information on 311,155,210 babies.   To facilitate development, I've sampled 100,000 lines in the file `babynames-100k.csv`.  During development, working with the sample.  Once done, set `sample` to `False` run your code and submit your notebook

In [1]:
sample = True
if sample == True:
    file_name = 'babynames-100k.csv'
else:
    file_name = 'babynames2018_state_gender_year_fname_number.csv'

#### Initialize

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://apache.claz.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [4]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext()

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
from pyspark.sql.types import *
from pyspark.sql.functions import *
Names = StructType([
                    StructField('state', StringType(), False),
                    StructField('gender', StringType(), False),
                    StructField('year', IntegerType(), False),
                    StructField('fname', StringType(), False),
                    StructField('number', IntegerType(), False)
])

In [6]:
from google.colab import files
uploaded = files.upload()

Saving babynames-100k.csv to babynames-100k.csv


### 1. read baby names

In [7]:
baby_names = spark.read.csv(file_name, schema=Names)
#lines = sc.textFile(file_name)
#baby_names = lines.map(lambda line: Names(*line.split(',')))
baby_names.take(5)

[Row(state='MD', gender='M', year=1964, fname='Linwood', number=11),
 Row(state='NC', gender='F', year=1983, fname='Jewel', number=9),
 Row(state='WA', gender='F', year=1990, fname='Eileen', number=7),
 Row(state='CA', gender='M', year=1994, fname='Emmet', number=10),
 Row(state='WY', gender='M', year=1997, fname='Jesse', number=16)]

### 2. count of male names

In [9]:
baby_names.createOrReplaceTempView('baby_names_v')

In [10]:
male_names = spark.sql('''
SELECT sum(number) as M_Count, fname as Name
FROM baby_names_v
WHERE gender='M'
GROUP BY fname
ORDER BY sum(number) DESC
''')

male_names.show(5)

+-------+-------+
|M_Count|   Name|
+-------+-------+
|  97088|  James|
|  87711|Michael|
|  73409|   John|
|  64740| Robert|
|  59591|  David|
+-------+-------+
only showing top 5 rows



### 3. count of female names

In [11]:
female_names = spark.sql('''
SELECT sum(number) as F_Count, fname as Name
FROM baby_names_v
WHERE gender='F'
GROUP BY fname
ORDER BY sum(number) DESC
''')

female_names.show(5)

+-------+--------+
|F_Count|    Name|
+-------+--------+
| 100955|    Mary|
|  29731|   Sarah|
|  26881|Jennifer|
|  26413|   Linda|
|  25237|Patricia|
+-------+--------+
only showing top 5 rows



### 3. gender neutral names

Please note that you join two RDDs on their keys i.e., the first element of the tuple.

In [12]:
neutral_names = male_names.join(female_names, male_names.Name == female_names.Name, 'inner')
neutral_names = neutral_names.select(male_names.Name, male_names.M_Count, female_names.F_Count)
#neutral_names = neutral_names.orderBy(male_names.M_Count.desc())
neutral_names.show(5)

+------+-------+-------+
|  Name|M_Count|F_Count|
+------+-------+-------+
| Tyler|   7582|    304|
| Tegan|     19|     21|
|Maddox|    154|      6|
| Ryder|    735|      5|
| Shawn|   4735|    322|
+------+-------+-------+
only showing top 5 rows



In [13]:
#neutral_names.take(5)

### 4. top 10 most popular neutral names

In [15]:
top10 = neutral_names.select(neutral_names.Name, round(neutral_names.M_Count/neutral_names.F_Count,2).alias("Ratio"), (neutral_names.M_Count+neutral_names.F_Count).alias("Total"))
top10 = top10.toDF('Name', 'Ratio', 'Total')

#Include "Total" to determine popularity of the names.

top10 = top10.where(top10.Ratio <= 4)
top10 = top10.where(top10.Ratio >= 0.25)
top10 = top10.orderBy(top10.Total.desc())

top10.show(10)

+------+-----+-----+
|  Name|Ratio|Total|
+------+-----+-----+
|Jordan| 1.88| 7766|
|Alexis|  0.7| 7356|
|Taylor| 0.32| 5951|
| Terry| 2.55| 5558|
|   Lee| 3.33| 4770|
|Leslie| 0.91| 4747|
| Angel| 1.43| 4543|
| Jamie| 0.43| 4521|
| Avery| 0.26| 4453|
|Jessie| 0.89| 4289|
+------+-----+-----+
only showing top 10 rows

